# 🧙‍♂️ D&D Meta Analysis (Mid-Course Milestone)

**Goal:** demonstrate the full data analysis workflow (load → inspect → clean → explore → visualize).

**Dataset:** `Data/dnd_classes_races_starter.csv` (hand-curated for this project; see citations below).

**Outputs in this notebook**
- Inspection: `.head()`, `.info()`, `.describe()`, missing values, duplicates  
- Cleaning: normalize casing, convert hit dice to numbers, map tier/popularity, drop dupes  
- EDA: counts, grouped summaries  
- Visuals: class bar chart, race pie chart, race × class heatmap  
- Derived table: race × class matrix (pivot)

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# show plots inline
%matplotlib inline

# project paths
BASE = Path().resolve()
DATA = (BASE / "Data")
DATA.mkdir(exist_ok=True)

CSV_IN      = DATA / "dnd_classes_races_starter.csv"
CLEAN_CSV   = DATA / "dnd_classes_races_clean.csv"
MATRIX_CSV  = DATA / "race_class_matrix.csv"
CLASS_BAR   = DATA / "class_counts.png"
RACE_PIE    = DATA / "race_pie.png"
HEATMAP_PNG = DATA / "race_class_heatmap.png"
SESSIONS    = DATA / "sessions_log.csv"

CSV_IN


## 1) Load data
If the CSV is missing, the notebook will raise an error. We’ll work from the `Data/` folder for reproducibility.

In [ ]:
if not CSV_IN.exists():
    raise FileNotFoundError(f"Missing dataset: {CSV_IN} — place your CSV in the Data/ folder.")

df = pd.read_csv(CSV_IN)
df.head()

## 2) Initial inspection
We check structure, types, basic stats, missingness, and duplicates to identify cleaning targets.

In [ ]:
display(df.info())            # structure & dtypes
display(df.describe(include="all"))  # summary stats incl. categoricals
display(df.isna().sum().rename("MissingValues"))
display(pd.Series({"DuplicateRows": int(df.duplicated().sum())}))

### Cleaning plan (based on inspection)
- Normalize **Race/Class/Subclass** casing to Title Case  
- Convert `'Hit Die'` strings like `d8` → numeric 8  
- Map **Tier (Power Level)** to ordinal (Low=1, Mid=2, High=3)  
- Map **Popularity** to ordinal (Low=1, Medium=2, High=3)  
- Drop exact duplicate rows

In [ ]:
df_clean = df.copy()

# normalize key categoricals
for col in ("Race", "Class", "Subclass"):
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip().str.title()

# convert Hit Die 'd8' -> 8
if "Hit Die" in df_clean.columns:
    num = (
        df_clean["Hit Die"]
        .astype(str).str.strip().str.lower().str.replace("d", "", regex=False)
    )
    df_clean["HitDieNum"] = pd.to_numeric(num, errors="coerce")

# map Tier/Popularity to ordered integers
if "Tier (Power Level)" in df_clean.columns:
    df_clean["TierNum"] = df_clean["Tier (Power Level)"].map({"Low": 1, "Mid": 2, "High": 3})

if "Popularity" in df_clean.columns:
    df_clean["PopularityScore"] = df_clean["Popularity"].map({"Low": 1, "Medium": 2, "High": 3})

# drop exact duplicates
before = len(df_clean)
df_clean = df_clean.drop_duplicates()
removed = before - len(df_clean)

removed, df_clean.head()


### Save the cleaned dataset (reproducibility)
This creates `Data/dnd_classes_races_clean.csv` so results are reproducible outside the notebook.

In [ ]:
df_clean.to_csv(CLEAN_CSV, index=False)
CLEAN_CSV

## 3) Exploratory Data Analysis (EDA)
Counts and quick groupings to understand distribution and relationships.

In [ ]:
class_counts = df_clean["Class"].value_counts().sort_values(ascending=False)
race_counts  = df_clean["Race"].value_counts().sort_values(ascending=False)

display(class_counts.rename("ClassCount"))
display(race_counts.rename("RaceCount"))

# optional: average numeric columns by class/race (if present)
summary_cols = [c for c in ("HitDieNum", "TierNum", "PopularityScore") if c in df_clean.columns]
if summary_cols:
    display(df_clean.groupby("Class")[summary_cols].mean().round(2).sort_index())
    display(df_clean.groupby("Race")[summary_cols].mean().round(2).sort_index())

## 4) Race × Class matrix (pivot)
This shows how many entries exist for each Race–Class combination.

In [ ]:
combo = pd.pivot_table(
    df_clean,
    index="Race",
    columns="Class",
    values="Subclass",  # any non-null column will do
    aggfunc="count",
    fill_value=0
).sort_index().reindex(sorted(df_clean["Class"].dropna().str.title().unique()), axis=1)

combo

In [ ]:
combo.to_csv(MATRIX_CSV)
MATRIX_CSV

## 5) Visualizations
Bar chart (classes), pie chart (races), and heatmap (Race × Class).

In [ ]:
ax = class_counts.plot(kind="bar", title="Class Counts")
ax.set_xlabel("Class"); ax.set_ylabel("Count")
plt.tight_layout()
plt.savefig(CLASS_BAR)
plt.show()

In [ ]:
ax = race_counts.plot(kind="pie", autopct="%1.1f%%", title="Race Distribution", ylabel="")
plt.tight_layout()
plt.savefig(RACE_PIE)
plt.show()

In [ ]:
plt.figure(figsize=(6.5, 4.5))
ax = plt.gca()
im = ax.imshow(combo, aspect="auto")
ax.set_xticks(range(len(combo.columns)))
ax.set_yticks(range(len(combo.index)))
ax.set_xticklabels(combo.columns, rotation=45, ha="right")
ax.set_yticklabels(combo.index)

# gridlines
ax.set_xticks([x-0.5 for x in range(1, len(combo.columns))], minor=True)
ax.set_yticks([y-0.5 for y in range(1, len(combo.index))], minor=True)
ax.grid(which="minor", linestyle="-", linewidth=0.5)
ax.tick_params(which="minor", bottom=False, left=False)

# cell labels
for i in range(len(combo.index)):
    for j in range(len(combo.columns)):
        ax.text(j, i, str(combo.iloc[i, j]), ha="center", va="center")

plt.title("Race × Class Matrix")
plt.colorbar(im, label="Count")
plt.tight_layout()
plt.savefig(HEATMAP_PNG)
plt.show()


## 6) (Optional) Sessions log merge
If `Data/sessions_log.csv` exists (even with a few rows), we can compare **what exists** vs **what’s actually played**.

Expected columns in sessions_log.csv:
`Date, Player, Campaign, Race, Class, Subclass`

In [ ]:
if SESSIONS.exists():
    sess = pd.read_csv(SESSIONS)
    # normalize keys for join
    for c in ("Race","Class","Subclass"):
        if c in sess.columns:
            sess[c] = sess[c].astype(str).str.strip().str.title()

    merged = sess.merge(
        df_clean[["Race","Class","Subclass"]].drop_duplicates(),
        on=["Race","Class","Subclass"],
        how="left",
        indicator=True
    )
    display(merged.head())
    print("Rows matched to reference options:", int((merged["_merge"]=="both").sum()))
else:
    print("No sessions_log.csv found; skipping merge demo.")

## 7) Insights (brief)
- Classes with highest counts: see **Class Counts** bar chart.  
- Race alignment: heatmap highlights stronger race–class pairings.  
- Popularity/Tier (if present as numeric) indicate player preference toward higher-power builds.  
- Next steps: expand dataset, analyze subclasses, and incorporate more play-session data for trends over time.

## 8) Citations / Sources

- **Primary data**: Hand-curated for educational purposes, inspired by Dungeons & Dragons 5e reference material.  
  - Wizards of the Coast — *Player’s Handbook* (5e).  
  - D&D System Reference Document (SRD).  
- **Author**: Sean (student, Code:You).

> Note: No copyrighted text was copied into the dataset; fields are short labels and categoricals for analysis only.
